In [1]:
import numpy as np 
import uproot as ur
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_curve, auc, recall_score, precision_score, RocCurveDisplay, PrecisionRecallDisplay
#from sklearn.utils import resample
#from utils import Logger, make_file_name, save_model, logloss_plot, auc_plot, aucpr_plot, roc_curve, pr_curve

In [2]:
random_seed = 135
sigfile = ur.open('train_sig_rare_lowQ_wPreselBDT_v6.root')
bkgfile = ur.open('train_bkg_data_sideBands_lowQ_wPreselBDT_v5.root')

In [14]:
features = ['Bprob', 'BsLxy', 'L2iso/L2pt', 'Bcos', 'Kiso/Kpt', 'LKdz', 'LKdr', 'Passymetry', 'Kip3d/Kip3dErr', 'L1id', 'L2id']
sample_weights = 'trig_wgt'
preselection = '(KLmassD0 > 2.) & ((Mll>1.05) & (Mll<2.45))'

sig_dict = sigfile['mytree'].arrays(features, library='np', cut=preselection)
bkg_dict = bkgfile['mytree'].arrays(features, library='np', cut=preselection)

In [ ]:
backgr = np.stack(list(bkg_dict.values()))
signal = np.stack(list(sig_dict.values()))

sig_weight = sigfile['mytree'].arrays(sample_weights, library='np', cut=preselection)[sample_weights]
bkg_weight = np.ones(backgr.shape[1])

X = np.transpose(np.concatenate((signal, backgr), axis=1))
y = np.concatenate((np.ones(signal.shape[1]), np.zeros(backgr.shape[1])))
weights = np.concatenate((sig_weight, bkg_weight))